# 🏍️ LossZero: Motorcycle Night Ride SegFormer-B2 Optimized

이 노트북은 **SegFormer-B2** 모델을 사용하여 야간 오토바이 주행 이미지의 시멘틱 세그멘테이션을 수행합니다.

### 🛠️ 주요 시나리오
- **모델**: SegFormer-B2 (Transformer 기반)
- **백본**: MiT-B2
- **사전 학습**: Cityscapes (도로 환경 특화)
- **최적화**: AdamW + FP16 Mixed Precision
- **손실 함수**: Weighted CrossEntropy (중요 객체 가중치 부여)

In [26]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm

print(f"PyTorch version: {torch.__version__}")


PyTorch version: 2.6.0


In [27]:
def get_device():
    if torch.cuda.is_available():
        return "cuda"
    # SegFormer의 permute 연산과 MPS 백엔드 간에 충돌
    # elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available() and torch.backends.mps.is_built():
    #     return "mps"
    return "cpu"

# ⚙️ 설정 (Configuration)
DATA_DIR = os.path.expanduser("~/Projects/LossZero/data/Motorcycle Night Ride Dataset") 
JSON_PATH = os.path.join(DATA_DIR, "COCO_motorcycle (pixel).json")
IMG_DIR = os.path.join(DATA_DIR, "images")

CFG = {
    "project": "LossZero",
    "model_name": "nvidia/segformer-b2-finetuned-cityscapes-1024-1024",
    "img_size": (352, 352),
    "batch_size": 4,
    "lr": 1e-4,
    "epochs": 25,
    "device": get_device()
}

print(f"Using device: {CFG['device']}")

Using device: cpu


In [28]:
def create_mask_from_json(coco, img_id, img_info, id_to_idx):
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)
    
    for ann in anns:
        cat_id = ann['category_id']
        if cat_id in id_to_idx:
            cls_idx = id_to_idx[cat_id]
            pixel_mask = coco.annToMask(ann)
            mask[pixel_mask == 1] = cls_idx
        
    return mask

def process_single_data(coco, img_id, img_dir, id_to_idx, transform=None):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(img_dir, img_info['file_name'])
    
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    mask = create_mask_from_json(coco, img_id, img_info, id_to_idx)
    
    if transform:
        augmented = transform(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']
    
    return image, torch.as_tensor(mask).long()

train_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

coco = COCO(JSON_PATH)
img_ids = list(coco.imgs.keys())
cat_ids = coco.getCatIds()
id_to_idx = {cat_id: i for i, cat_id in enumerate(cat_ids)}
print(f"Category Mapping: {id_to_idx}")

loading annotations into memory...
Done (t=0.79s)
creating index...
index created!
Category Mapping: {1329681: 0, 1323885: 1, 1323884: 2, 1323882: 3, 1323881: 4, 1323880: 5}


In [29]:
id2label = {i: coco.loadCats(cat_id)[0]['name'] for cat_id, i in id_to_idx.items()}
label2id = {v: k for k, v in id2label.items()}

model = SegformerForSemanticSegmentation.from_pretrained(
    CFG['model_name'],
    num_labels=len(id_to_idx),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to(CFG['device'])

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'], weight_decay=0.01)

# ⚖️ 클래스별 가중치 설정 (Class Weights)
# 0: Rider(2.0), 1: My bike(2.0), 2: Moveable(4.0), 3: Lane Mark(8.0), 4: Road(1.0), 5: Undrivable(1.0)
# 면적이 좁고 안전에 핵심적인 '차선(Lane Mark, 8x)'과 '장애물(Moveable, 4x)'에 높은 가중치 부여
weights = torch.tensor([2.0, 2.0, 4.0, 8.0, 1.0, 1.0], dtype=torch.float).to(CFG['device'])
criterion = nn.CrossEntropyLoss(weight=weights)

scaler = GradScaler('cuda') if CFG['device'] == 'cuda' else None

Loading weights:   0%|          | 0/380 [00:00<?, ?it/s]

SegformerForSemanticSegmentation LOAD REPORT from: nvidia/segformer-b2-finetuned-cityscapes-1024-1024
Key                           | Status   |                                                                                                    
------------------------------+----------+----------------------------------------------------------------------------------------------------
decode_head.classifier.bias   | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19]) vs model:torch.Size([6])                      
decode_head.classifier.weight | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19, 768, 1, 1]) vs model:torch.Size([6, 768, 1, 1])

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight shapes.


In [ ]:
print("🚀 SegFormer-B2 Training Start...")

for epoch in range(CFG['epochs']):
    model.train()
    np.random.shuffle(img_ids)
    epoch_loss = 0
    
    pbar = tqdm(range(0, len(img_ids), CFG['batch_size']), desc=f"Epoch {epoch+1}")
    for i in pbar:
        batch_ids = img_ids[i : i + CFG['batch_size']]
        
        images, masks = [], []
        for img_id in batch_ids:
            img, msk = process_single_data(coco, img_id, IMG_DIR, id_to_idx, train_transform)
            images.append(img)
            masks.append(msk)
            
        # Force contiguous input
        X = torch.stack(images).to(CFG['device']).contiguous()
        y = torch.stack(masks).to(CFG['device']).contiguous()
        
        optimizer.zero_grad()
        
        if scaler:
            with autocast('cuda'):
                # Force contiguous output
                outputs = model(X).logits.contiguous()
                upsampled_logits = nn.functional.interpolate(
                    outputs, 
                    size=y.shape[-2:], 
                    mode="bilinear", 
                    align_corners=False
                ).contiguous()
                loss = criterion(upsampled_logits, y)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            # Force contiguous output
            outputs = model(X).logits.contiguous()
            upsampled_logits = nn.functional.interpolate(
                outputs, 
                size=y.shape[-2:], 
                mode="bilinear", 
                align_corners=False
            ).contiguous()
            loss = criterion(upsampled_logits, y)
            loss.backward()
            optimizer.step()
        
        epoch_loss += loss.item()
        pbar.set_postfix(loss=loss.item())
        
    avg_loss = epoch_loss / (len(img_ids) // CFG['batch_size'])
    print(f"Epoch [{epoch+1}/{CFG['epochs']}] Avg Loss: {avg_loss:.4f}")

🚀 SegFormer-B2 Training Start...


Epoch 1:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/25] Avg Loss: 1.1270


Epoch 2:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [2/25] Avg Loss: 0.5323


Epoch 3:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [3/25] Avg Loss: 0.3795


Epoch 4:   0%|          | 0/50 [00:00<?, ?it/s]